In [3]:
from importlib import reload
from dotenv import load_dotenv

load_dotenv(r"C:\Users\vireima\tie-dashboard\.env")

import os

# print(os.getenv("SEVERA_CLIENT_SCOPE"))

import pandas as pd
import altair as alt

import src.severa.base_client
import src.severa.fetch
import src.database
from src.daterange import DateRange
import datetime

reload(src.severa.base_client)
reload(src.severa.fetch)
reload(src.database)

span = DateRange(540)
alt.data_transformers.disable_max_rows()

# async with src.severa.fetch.Fetcher() as f:
#    df = await f.get_resource_allocations(span)

DataTransformerRegistry.enable('default')

In [3]:
async with src.severa.fetch.Fetcher() as f:
    df2 = await f.get_allocations_with_maxes(span)

df2.head()

2023-06-02 07:44:39.076 | SUCCESS  | src.severa.base_client:get_with_retries:126 - HTTP/1.1 GET users [0]: Response (9) in 0.27s.
2023-06-02 07:44:39.334 | SUCCESS  | src.severa.base_client:get_with_retries:126 - HTTP/1.1 GET activities [0]: Response (0) in 0.23s.
2023-06-02 07:44:39.452 | SUCCESS  | src.severa.base_client:get_with_retries:126 - HTTP/1.1 GET activities [0]: Response (20) in 0.36s.
2023-06-02 07:44:39.488 | SUCCESS  | src.severa.base_client:get_with_retries:126 - HTTP/1.1 GET activities [0]: Response (1) in 0.39s.
2023-06-02 07:44:39.496 | SUCCESS  | src.severa.base_client:get_with_retries:126 - HTTP/1.1 GET activities [0]: Response (0) in 0.41s.
2023-06-02 07:44:39.661 | SUCCESS  | src.severa.base_client:get_with_retries:126 - HTTP/1.1 GET activities [0]: Response (1) in 0.16s.
2023-06-02 07:44:39.692 | SUCCESS  | src.severa.base_client:get_with_retries:126 - HTTP/1.1 GET activities [0]: Response (0) in 0.59s.
2023-06-02 07:44:39.711 | SUCCESS  | src.severa.base_client

,businessunit-user,value,user,project,phase,date,id,forecast-date,type
96,TIE,0.642713,eb8d73da-8c1c-7996-b085-332e777dff1a,24e8dd29-d61b-c481-5013-0ff872b54c5d,979f7378-a78f-5d9a-2498-ef3550b71aa4,2023-06-02 00:00:00+00:00,allocation,2023-06-03 00:00:00+00:00,external
97,TIE,0.642713,eb8d73da-8c1c-7996-b085-332e777dff1a,24e8dd29-d61b-c481-5013-0ff872b54c5d,979f7378-a78f-5d9a-2498-ef3550b71aa4,2023-06-02 00:00:00+00:00,allocation,2023-06-04 00:00:00+00:00,external
98,TIE,0.642713,eb8d73da-8c1c-7996-b085-332e777dff1a,24e8dd29-d61b-c481-5013-0ff872b54c5d,979f7378-a78f-5d9a-2498-ef3550b71aa4,2023-06-02 00:00:00+00:00,allocation,2023-06-05 00:00:00+00:00,external
99,TIE,0.642713,eb8d73da-8c1c-7996-b085-332e777dff1a,24e8dd29-d61b-c481-5013-0ff872b54c5d,979f7378-a78f-5d9a-2498-ef3550b71aa4,2023-06-02 00:00:00+00:00,allocation,2023-06-06 00:00:00+00:00,external
100,TIE,0.642713,eb8d73da-8c1c-7996-b085-332e777dff1a,24e8dd29-d61b-c481-5013-0ff872b54c5d,979f7378-a78f-5d9a-2498-ef3550b71aa4,2023-06-02 00:00:00+00:00,allocation,2023-06-07 00:00:00+00:00,external


In [7]:
data = src.database.Base("kpi-dev", "allocations").find()

2023-06-02 21:05:19.811 | INFO     | src.database:find:32 - Query '{}' resulted in 87536 results in 10.50s.


In [4]:
FI_LOCALE_JSON = {
    "dateTime": "%A, %-d. %Bta %Y klo %X",
    "date": "%-d.%-m.%Y",
    "time": "%H:%M:%S",
    "periods": ["a.m.", "p.m."],
    "days": [
        "sunnuntai",
        "maanantai",
        "tiistai",
        "keskiviikko",
        "torstai",
        "perjantai",
        "lauantai",
    ],
    "shortDays": ["Su", "Ma", "Ti", "Ke", "To", "Pe", "La"],
    "months": [
        "tammikuu",
        "helmikuu",
        "maaliskuu",
        "huhtikuu",
        "toukokuu",
        "kesäkuu",
        "heinäkuu",
        "elokuu",
        "syyskuu",
        "lokakuu",
        "marraskuu",
        "joulukuu",
    ],
    "shortMonths": [
        "Tammi",
        "Helmi",
        "Maalis",
        "Huhti",
        "Touko",
        "Kesä",
        "Heinä",
        "Elo",
        "Syys",
        "Loka",
        "Marras",
        "Joulu",
    ],
}
alt.renderers.set_embed_options(timeFormatLocale=FI_LOCALE_JSON)

RendererRegistry.enable('default')

In [5]:
users = await src.severa.fetch.Fetcher().users()

2023-06-02 21:04:44.832 | SUCCESS  | src.severa.base_client:get_with_retries:126 - HTTP/1.1 GET users [0]: Response (9) in 0.31s.


In [11]:
users_df = pd.DataFrame([{"user": u.guid, "name": u.firstName} for u in users])

source = (
    data[(data["date"] == data["date"].max()) & (data["type"] != "max")]
    .groupby(["forecast-date", "user"])["value"]
    .sum()
    .reset_index()
).merge(users_df, on="user")

source["week"] = source["forecast-date"].dt.isocalendar().week

source

,forecast-date,user,value,name,week
0,2023-06-03,0d100bc1-8376-7ac5-7bbb-055cdc20497d,1.811146,Johanna,22
1,2023-06-04,0d100bc1-8376-7ac5-7bbb-055cdc20497d,1.811146,Johanna,22
2,2023-06-05,0d100bc1-8376-7ac5-7bbb-055cdc20497d,1.811146,Johanna,23
3,2023-06-06,0d100bc1-8376-7ac5-7bbb-055cdc20497d,1.811146,Johanna,23
4,2023-06-07,0d100bc1-8376-7ac5-7bbb-055cdc20497d,1.811146,Johanna,23
...,...,...,...,...,...
3777,2024-03-25,eb8d73da-8c1c-7996-b085-332e777dff1a,0.285124,Vladimirs,13
3778,2024-03-26,eb8d73da-8c1c-7996-b085-332e777dff1a,0.285124,Vladimirs,13
3779,2024-03-27,eb8d73da-8c1c-7996-b085-332e777dff1a,0.285124,Vladimirs,13
3780,2024-03-28,eb8d73da-8c1c-7996-b085-332e777dff1a,0.285124,Vladimirs,13


In [12]:
options = ["d", "week", "m"]
labels = [option + " " for option in options]

input_dropdown = alt.binding_radio(
    # Add the empty selection which shows all when clicked
    options=options + [None],
    labels=labels + ["All"],
    name="Region: ",
)
selection = alt.selection_point(
    fields=["day", "week", "month"],
    bind=input_dropdown,
)


cond = alt.param(0.5)

brush = alt.selection_interval(encodings=["x"])

base = (
    alt.Chart(source)
    # .encode(x="forecast-date:T", y="sum(value):Q")
    .properties(width=900, height=200)
)

upper = base.mark_area().encode(
    x=alt.X(
        field=alt.condition(
            cond < 0.8, "yearmonth(forecast-date)", "yearmonthdate(forecast-date)"
        ),
        type="temporal",
    ).scale(domain=brush),
    y="sum(value):Q",
    color="name:N",
    # tooltip=[
    #    alt.Tooltip("forecast-date", title="Päivämäärä", format="%-d.%-m.%Y"),
    #    alt.Tooltip("name", title="Nimi"),
    #    alt.Tooltip("value", title="h/vrk", format=".1f")]
    # ).transform_calculate(
    #    month = "month(date)",
)

lower = (
    base.mark_area()
    .encode(x="forecast-date:T", y="sum(value):Q")
    .properties(height=60)
    .add_params(brush)
)

upper & lower

SchemaValidationError: '0.5' is an invalid value for `name`. Valid values are of type 'string'.

In [30]:
from dotenv import load_dotenv

load_dotenv(r"C:\Users\vireima\tie-dashboard\.env")

import src.database

reload(src.database)

b = src.database.Base("kpi-dev", "allocations")

In [31]:
x = b.find()

2023-05-31 09:01:45.483 | INFO     | src.database:find:32 - Query '{}' resulted in 23438 results in 12.39s.


In [32]:
x.loc[:, ["forecast-date", "value"]]

,forecast-date,value
0,2023-02-27,0.642713
1,2023-02-28,0.642713
2,2023-03-01,0.642713
3,2023-03-02,0.642713
4,2023-03-03,0.642713
...,...,...
23433,2023-06-26,0.333333
23434,2023-06-27,0.333333
23435,2023-06-28,0.333333
23436,2023-06-29,0.333333


In [227]:
# Quick mock data
a = pd.Series(1, index=pd.date_range("2023-01-01", "2023-01-05", freq="D"))
b = pd.Series(1, index=pd.date_range("2023-01-03", "2023-01-08", freq="D"))
c = pd.Series(1, index=pd.date_range("2023-01-04", "2023-01-12", freq="D"))

df = pd.DataFrame({"user": ["A", "B", "B"], "unit": ["TIE"] * 3, "values": [a, b, c]})


# Add series with datetime indices together
def combine_series(df):
    return reduce(partial(pd.Series.add, fill_value=0), df)


# This works nicely...
# ok = combine_series(df["values"])

# ...outputs a DF, indices are OK:
# 2023-01-01    1.0
# 2023-01-02    1.0
# 2023-01-03    2.0
# 2023-01-04    3.0
# ...
# 2023-01-12    1.0
# Freq: D, dtype: float64

# This 'works' in that the values are summed correctly,
# but the indices are gone now
# not_ok = df.groupby(["unit", "user"], as_index=False).agg(combine_series)

# This is seemingly a np array
# not_ok.loc["B", "values"]
# array([1., 2., 2., 2., 2., 2., 1., 1., 1., 1.])

grouping = ["unit", "user"]
grouped = df.groupby(grouping)

pd.DataFrame(
    {"values": combine_series(v["values"])} | dict(zip(grouping, k)) for k, v in grouped
)

,values,unit,user
0,2023-01-01 1 2023-01-02 1 2023-01-03 ...,TIE,A
1,2023-01-03 1.0 2023-01-04 2.0 2023-01-05...,TIE,B


In [298]:


hours_per_day = 1.5
index=pd.date_range(
    "2023-05-01",
    "2023-07-30",
    freq="D",
)
pd.DataFrame(index=index, data={"date":"2023-05-20", "user": "ville","value":hours_per_day})[]

,date,user,value
2023-05-01,2023-05-20,ville,1.5
2023-05-02,2023-05-20,ville,1.5
2023-05-03,2023-05-20,ville,1.5
2023-05-04,2023-05-20,ville,1.5
2023-05-05,2023-05-20,ville,1.5
...,...,...,...
2023-07-26,2023-05-20,ville,1.5
2023-07-27,2023-05-20,ville,1.5
2023-07-28,2023-05-20,ville,1.5
2023-07-29,2023-05-20,ville,1.5
